In [18]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from app.database import SessionLocal, Base, engine
from app.models import *
from app import models
from app.utils import hash_password
from app.db_utils import (
    reset_db,
    drop_table, 
    to_df, 
    get_table_names, 
    count_records, 
    backup_table,
    get_table_schema,
    get_session,
    execute_raw_query
)
import random
import uuid
from typing import List, Dict, Set, Tuple
from devseed import gather_unique_cf_handles
from app.codeforces_api import CodeforcesAPI
import numpy as np
from faker import Faker
from app.models import (
    User,
    Group,
    GroupMembership,
    Report,
    Contest,
    ContestParticipation,
    Role,
)
from sqlalchemy import func

def random_subset(n, k):
    return random.sample([i for i in range(n)], k)

def pick_random_subset(population: list, k: int) -> list:
    """k distinct elements from population (k may be 0)."""
    if k == 0:
        return []
    return random.sample(population, k)

def get_mock_contest_standings(cid, include_users=None):
    if include_users is None:
        include_users = []
    res = {
        'contest': {
            'id': cid,
            'name': f'Codeforces Round {cid}',
            'type': 'CF',
            'phase': 'FINISHED',
            'frozen': False,
            'durationSeconds': 7200,
            'startTimeSeconds': 1746110100,
            'relativeTimeSeconds': 267545
        },
        'standings': [
        ]
    }
    num_participants = 8000
    standings = [
        {'handle': 'random-user'+str(i+1), 'rank': (i+1), 'points': 9754 - i, 'penalty': 0}
        for i in range(num_participants)
    ]
    res['standings'] = standings
    udx = pick_random_subset([i for i in range(num_participants)], len(include_users))

    for i in range(len(udx)):
        res['standings'][udx[i]]['handle'] = include_users[i]
    
    return list(set(res))

In [19]:
cf_handles = gather_unique_cf_handles([2102, 2101, 2109, 2107])


» cf api → contest 2102

» cf api → contest 2101

» cf api → contest 2109

» cf api → contest 2107

» cf handles gathered ➜ 5332 unique


In [20]:
SEED = 88
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
faker = Faker()
DEFAULT_PASS = "devpass"

Base.metadata.create_all(bind=engine)
db = SessionLocal()

cf_api = CodeforcesAPI()

In [21]:
admin_users = [
    User(
        user_id="shrey",
        role=Role.admin,
        cf_handle="misaki",
        email_id="talmudlover@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
    User(
        user_id="ani",
        role=Role.admin,
        cf_handle="roomTemperatureIQ",
        email_id="ecwcejhvchv@gmail.com",
        atcoder_handle=None,
        codechef_handle=None,
        twitter_handle=None,
        trusted_score=88,
        hashed_password=hash_password(DEFAULT_PASS),
    ),
]

users = admin_users

for handle in cf_handles:
    uid = 'testUser' + str(len(users)-1)
    users.append(
        User(
            user_id=uid,
            role=Role.user,
            cf_handle=handle,
            atcoder_handle=None if random.random() < 0.5 else uid + "_ac",
            codechef_handle=None if random.random() < 0.7 else uid + "_cc",
            twitter_handle=None if random.random() < 0.6 else uid + "_tw",
            trusted_score=random.randint(0, 100),
            email_id=str(uid) + '@gmail.com',
            hashed_password=hash_password(DEFAULT_PASS),
        )
    )

len(users)

5334

In [22]:
# POPULATE GROUPS
NUM_GROUPS = 30
common_group = Group(
    group_id="main",
    group_name="main",
    group_description="group consisting of ALL users",
    is_private=False,
)

groups = [common_group]
for g_idx in range(1, NUM_GROUPS):
    g_id = f"g{g_idx:02d}"
    g_name = faker.unique.catch_phrase().lower().replace(" ", "‑")
    is_priv = random.random() < 0.3
    group = Group(
        group_id=g_id,
        group_name=g_name,
        group_description=faker.sentence(nb_words=10),
        is_private=is_priv,
    )
    groups.append(group)


In [23]:
# POPULATE MEMBERSHIPS
memberships = []


# common group should have ALL users
memberships.append(GroupMembership(
    user_id=users[0].user_id,
    group_id=groups[0].group_id,
    role=Role.admin,
    user_group_rating=1500,
    user_group_max_rating=1500,
))

for i in range(1, len(users)):
    mxr = 1500
    cur = 1500
    memberships.append(
        GroupMembership(
            user_id=users[i].user_id,
            group_id=groups[0].group_id,
            role=Role.user,
            user_group_rating=cur,
            user_group_max_rating=mxr,
        )
    )

# add memberships to other groups
size_palette = [5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597]
while len(size_palette) < NUM_GROUPS - 1:
    size_palette.append(random.randint(10, 800))
random.shuffle(size_palette)

user_idx = [i for i in range(len(users))]

for i in range(1, NUM_GROUPS):
    grp_size = size_palette[i-1]
    members = pick_random_subset(user_idx, grp_size)
    # admin for this group
    memberships.append(
        GroupMembership(
            user_id = users[members[0]].user_id,
            group_id = groups[i].group_id,
            role=Role.admin,
            user_group_rating=1500,
            user_group_max_rating=1500, 
        )
    )
    for j in members[1:]:
        memberships.append(
            GroupMembership(
                user_id=users[j].user_id,
                group_id=groups[i].group_id,
                role=Role.user,
                user_group_rating=1500,
                user_group_max_rating=1500,
            )
        )

# memberships

In [24]:
reset_db()
# db.rollback()

db.add_all(users)
db.add_all(groups)
db.add_all(memberships)

dropping all tables...
all tables dropped.
creating tables from models...
schema rebuilt.


In [25]:
db.commit()

In [12]:
# populate CONTESTS]
cf_contests = [2102, 2101, 2109, 2107]
contests = []
for i in range(len(cf_contests)):
    standingsObj = cf_api.contest_standings(cf_contests[i])
    contests.append(
        Contest(
            contest_id = f"c{i}",
            contest_name = standingsObj['contest']['name'],
            platform = "Codeforces",
            start_time_posix = standingsObj['contest']['startTimeSeconds'],
            duration_seconds = standingsObj['contest']['durationSeconds'],
            link = f'https://codeforces.com/contest/{cf_contests[i]}',
            finished=False,
            internal_contest_identifier = cf_contests[i],
        )
    )

In [16]:
# populate contest participations
participations = []

for contest in contests:
    for group in groups:
        members = group.memberships
        n_members = len(members)
        parts = random_subset(n_members, int(3*n_members//4))
        for ii in parts:
            participations.append(
                ContestParticipation(
                    user_id = members[ii].user_id,
                    group_id = group.group_id,
                    contest_id = contest.contest_id,
                    rating_before = members[ii].user_group_rating,
                )
            )
